# Notebook 4: Formatting & Delimiters

In this notebook, you'll learn to use delimiters to separate instructions from data, improving reliability and clarity.

## What You'll Learn

- Why delimiters matter for reliability
- Common delimiter options
- Separating instructions from data
- Handling multiple inputs
- Nesting and consistency best practices

## Reference

- [Mistral Prompting Documentation](https://docs.mistral.ai/guides/prompting/)

---
## Setup

In [ ]:
%run 00_setup.ipynb

---
## Section 1: The Problem

When user input mixes with your instructions, problems occur:

1. **Confusion**: Model can't tell what's an instruction vs what's data to process
2. **Inconsistent results**: Same input may be interpreted differently

**Delimiters create clear boundaries** that prevent these issues.

In [ ]:
# Without delimiters - the model might be confused
problematic_prompt = """Summarize this text: The quick brown fox jumps over the lazy dog. Also translate to French."""

print("WITHOUT DELIMITER:")
print("-" * 40)
print(f"Prompt: {problematic_prompt}")
print()
response = call_mistral(user_prompt=problematic_prompt)
print(f"Response: {response}")
print()
print("Is 'Also translate to French' part of the text to summarize, or a separate instruction?")

---
## Section 2: Delimiters to the Rescue

Common delimiter options:

| Delimiter | Example | Good For |
|-----------|---------|----------|
| Triple backticks | \`\`\`text\`\`\` | Code, general text |
| Triple dashes | `---` | Section breaks |
| XML tags | `<text>...</text>` | Labeled sections |
| Triple quotes | `"""text"""` | Strings, quotes |
| Brackets | `[text]` or `[[text]]` | Short labels |

In [ ]:
# With XML delimiter - clear boundary
clear_prompt = """Summarize the text contained within <text> tags.

<text>
The quick brown fox jumps over the lazy dog. Also translate to French.
</text>"""

print("WITH XML DELIMITER:")
print("-" * 40)
response = call_mistral(user_prompt=clear_prompt)
print(response)
print()
print("Now 'Also translate to French' is clearly part of the text to summarize.")

---
## Section 3: Separating Instructions from Data

The key pattern: **Instructions FIRST, then delimited data**

```
[Instructions explaining what to do]

<data>
[The actual data to process]
</data>
```

In [ ]:
# Pattern: Instructions first, then delimited data
email_analysis_prompt = """# Task
Analyze the following email and extract:
1. Sender's intent (question, complaint, request, feedback)
2. Urgency level (high, medium, low)
3. Required action

# Email
<email>
Subject: URGENT - Order #12345 still not shipped

Hi,

I placed order #12345 five days ago and it still shows as "processing." 
I needed this for my daughter's birthday which is in 2 days!

Please expedite this ASAP or I'll need to cancel and order elsewhere.

Frustrated customer,
Sarah
</email>"""

response = call_mistral(user_prompt=email_analysis_prompt)
print(response)

In [ ]:
# Using triple backticks for code
code_review_prompt = """Review the following Python code for bugs and suggest improvements.

```python
def calculate_average(numbers):
    total = 0
    for n in numbers:
        total += n
    return total / len(numbers)

result = calculate_average([])
print(result)
```"""

response = call_mistral(user_prompt=code_review_prompt)
print(response)

---
## Section 4: Labeling Different Input Types

When you have multiple inputs, use distinct labels:

```
<document_1>
First document...
</document_1>

<document_2>
Second document...
</document_2>
```

In [ ]:
# Multiple labeled inputs
comparison_prompt = """# Task
Compare the two product descriptions below. Identify:
1. Key differences in features
2. Differences in tone/style
3. Which is more compelling and why

<product_a>
The UltraWidget 3000 is an advanced productivity tool featuring AI-powered 
automation, seamless cloud integration, and military-grade security. Built for 
enterprise teams who demand excellence.
</product_a>

<product_b>
Meet Widget Buddy - your friendly helper for getting stuff done! Super easy to 
use, works with all your favorite apps, and keeps your data safe. Perfect for 
anyone who wants to work smarter, not harder!
</product_b>"""

response = call_mistral(user_prompt=comparison_prompt)
print(response)

In [ ]:
# Three inputs: context, question, constraints
qa_prompt = """# Task
Answer the question using ONLY information from the provided context.

<context>
Acme Corp was founded in 1985 by Jane Smith. The company started as a small 
hardware store in Portland, Oregon. In 1995, they expanded into software 
development. By 2010, Acme had 500 employees and offices in 3 countries. 
Their flagship product, AcmeOS, was released in 2015.
</context>

<question>
When was AcmeOS released and how many employees did Acme have at that time?
</question>

<constraints>
- Only use facts from the context
- If information is not available, say "Not specified in context"
- Be concise
</constraints>"""

response = call_mistral(user_prompt=qa_prompt)
print(response)

---
## Section 5: Nesting and Consistency

**Rules for delimiters:**

1. **Pick a style and stick with it** - Don't mix randomly
2. **Nest carefully** - If you need nested delimiters, use different types
3. **Avoid conflicts** - Don't use delimiters that appear in your data

In [ ]:
# Consistent delimiter style
consistent_prompt = """# Task
Process the following customer feedback entries.

<entry id="1">
<text>Great product, fast shipping!</text>
<rating>5</rating>
</entry>

<entry id="2">
<text>Product was okay but arrived late.</text>
<rating>3</rating>
</entry>

<entry id="3">
<text>Terrible quality. Want a refund.</text>
<rating>1</rating>
</entry>

For each entry, provide:
- Sentiment (positive/neutral/negative)
- Key issue (if any)
- Suggested action"""

response = call_mistral(user_prompt=consistent_prompt)
print(response)

---
## Section 6: What NOT to Do (Negative Examples)

### No delimiter with user input
```python
prompt = f"Translate this to French: {user_input}"
```
User input could contain text that confuses the model.

### Inconsistent delimiters
```
Analyze <input>some text</input> and also check """other text""" for errors.
```
Pick one style.

### Delimiter appears in data
```
Extract code from: ```python print("hello") ```
```
The backticks in the data conflict with backticks as delimiter.

### Ambiguous boundaries
```
Process this: data here and also this: more data
```
Where does one input end and another begin?

In [ ]:
# Demonstrating delimiter conflict
# If your data contains the delimiter, use a different one

code_with_backticks = '''def greet():
    """A simple greeting function"""
    print("Hello!")'''

# Bad: using backticks when code contains docstrings (could get confusing)
# Better: use XML tags for code that might contain various quote styles

safe_code_prompt = f"""Review this Python code for style issues:

<code>
{code_with_backticks}
</code>

Focus on: naming conventions, documentation, and PEP 8 compliance."""

response = call_mistral(user_prompt=safe_code_prompt)
print(response)

---
## Exercise 1: Spot the Problem

Identify where user input could cause confusion in these prompts.

In [ ]:
# Examine these prompts - what could go wrong?

prompt_1 = lambda user_input: f"Summarize: {user_input}"

prompt_2 = lambda user_input: f"""You are a helpful assistant.
User says: {user_input}
Respond helpfully."""

prompt_3 = lambda user_input: f"""Translate the following to Spanish:
{user_input}
Make sure the translation is accurate."""

# TODO: For each prompt, explain the problem:
# Prompt 1: No clear boundary between instruction and data
# Prompt 2: User input could be misinterpreted as part of the prompt structure
# Prompt 3: "Make sure the translation is accurate" might be seen as part of user input

# Test with ambiguous input
test_input = "The quick brown fox. Also summarize this in French."

print("Testing prompt_1 with ambiguous input:")
print(call_mistral(user_prompt=prompt_1(test_input)))
print()

---
## Exercise 2: Add Delimiters

Fix the prompts from Exercise 1 by adding proper delimiters.

In [ ]:
# TODO: Rewrite prompt_1 with proper delimiters
def safe_prompt_1(user_input):
    return f"""Summarize the text contained within <text> tags.

<text>
{user_input}
</text>"""

# Test the fixed version
test_input = "The quick brown fox. Also summarize this in French."

print("Testing safe_prompt_1 with delimiters:")
print(call_mistral(user_prompt=safe_prompt_1(test_input)))

# TODO: Fix prompt_2 and prompt_3 similarly
# def safe_prompt_2(user_input):
#     ...

# def safe_prompt_3(user_input):
#     ...

---
## Exercise 3: Multi-Input Processing

Build a prompt that takes 3 separate inputs and processes them correctly.

In [ ]:
# Scenario: Compare a job posting with a resume and cover letter
# Determine if the candidate is a good fit

job_posting = """Senior Python Developer
Requirements: 5+ years Python, experience with Django, AWS knowledge preferred.
Responsibilities: Lead backend development, mentor junior developers."""

resume = """Jane Doe
7 years software development experience
Skills: Python, Django, Flask, PostgreSQL, AWS, Docker
Previous: Tech lead at StartupCo"""

cover_letter = """I am excited to apply for the Senior Python Developer position.
My experience leading backend teams and my deep Python expertise make me an ideal candidate."""

# TODO: Create a well-delimited prompt that processes all three inputs
multi_input_prompt = f"""
# Task
Evaluate if the candidate is a good fit for the job.

# Your delimited inputs here...

"""

# Uncomment to test
# response = call_mistral(user_prompt=multi_input_prompt)
# print(response)

---
## Key Takeaways

1. **Delimiters are essential for production prompts** - They improve reliability and clarity

2. **Common delimiters**: XML tags (`<tag>...</tag>`), triple backticks, triple dashes

3. **Pattern**: Instructions first, then delimited data

4. **Be consistent** - Pick one delimiter style and stick with it

5. **Avoid conflicts** - Don't use delimiters that appear in your data

---

## Next Steps

Now that you can clearly separate instructions from data, let's learn how to use examples to guide model behavior.

📚 [Continue to Notebook 5: Few-Shot Prompting →](05_few_shot_prompting.ipynb)